# 2025년 5월 19일 금요일 (17일차)
join을 겨우 이해했다. 서브쿼리부터 이해할 수 없었다. 아무리 봐도 왜 이렇게 되는지 모르겠다... 힘들다. 다시 공부해야겠다 ㅜㅜ


<br><br><br>

# 📜 목차
- MySQL
  - 제약조건
    - Primary Key
    - Foreign Key
  - join
    - inner join
    - left join
  - SubQuery (하위 질의)
    - correlated subquery (상관 서브쿼리)
  - Group By
    

<br><br><br><br>

# 🟩 DB SQL with MySQL (2일차)

<br><br><br><br>

# 🟩 제약조건 

### 🟡 primary key 
- 중복 안되고 Null값을 갖지 않아야 한다. 이 조건을 만족해야만 promary key를 줄 수 있다.
- ex) 회원 아이디 - 예전에는 회원아이디를 primary key로 한다.. 수정불가
  - 아이디 변경 가능했던 사이트들은 별도의 일련번호를 부여해서 그 일련번호를 primary로 하고, 회원 아이디를 unique-key를 준다.
  - unique 제약조건은 중복은 허용하지 않지만, null값은 허용
- 결론적으로 테이블의 primary key는 하나만 줄 수 있다. 
  - 하지만 여러개의 필드를 묶어서 하나의 primary key를 지정하는 경우도 있다.
  
| 항목              | 설명                                                                                                                                                                   |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 정의              | 테이블에서 각 행(row)을 **고유하게 식별**할 수 있는 컬럼 또는 컬럼들의 집합                                                                                                                      |
| 조건              | <ul><li>**중복 불가**</li><li>**NULL 불가**</li></ul> 이 두 가지 조건을 반드시 만족해야 Primary Key로 지정 가능                                                                               |
| 개수 제한           | 테이블당 **Primary Key는 1개만 지정 가능**                                                                                                                                      |
| 복합 키            | 2개 이상의 컬럼을 묶어 하나의 Primary Key로 설정할 수 있음 (복합 기본키, Composite Key)                                                                                                      |
| 예시              | <ul><li>회원 테이블에서 `회원아이디`를 Primary Key로 설정하는 경우가 많음</li><li>하지만 아이디를 변경 가능하게 하고 싶다면, \*\*내부 식별용 일련번호(id)\*\*를 Primary Key로 설정하고, `회원아이디`는 `UNIQUE` 제약조건을 부여</li></ul> |
| Unique 제약조건과 차이 | <ul><li>**UNIQUE**: 중복 불가하지만, **NULL은 허용됨**</li><li>**PRIMARY KEY**: 중복도 안 되고, NULL도 안 됨</li></ul>                                                                   |
| 수정 가능성 고려       | Primary Key는 **가능하면 수정되지 않는 값**을 선택해야 함<br>(회원아이디는 변경될 수 있으므로 내부 id를 사용하는 게 일반적)                                                                                     |


In [ ]:
-- < primary key >
-- emp 테이블의 정보를 확인하고 primary key가 존재하면 삭제하려고 한다.
USE mydb;

SELECT * FROM emp LIMIT 10;
DESC emp;  -- empno 가 PRIMARY임

INSERT INTO emp(empno, ename) VALUES (8011, '장길산');

ALTER TABLE emp DROP PRIMARY KEY;

DESC emp;  -- empno 가 PRIMARY임
INSERT INTO emp(empno, ename) VALUES (8011, '장길산');

SELECT * FROM emp;
 
-- primary key 지정은 불가능하다. 이유: empno 필드갑이 이미 중복 존
ALTER TABLE emp ADD CONSTRAINT pk_emp primary KEY (empno)
-- emp 테이블의 기본 키로 설정된 컬럼(예: empno)에 이미 8011이라는 값이 존재하는데,
-- 다시 동일한 값을 넣으려 해서 발생했습니다.

DELETE FROM emp WHERE empno = 8011 AND ename = '장길산';

SELECT * FROM emp;

ALTER TABLE emp ADD CONSTRAINT pk_emp PRIMARY key(empno);

-- 갑자기 설정을 변경???  edit - preference - sql editor - 하단에 safe updates 체크 해제하고 재시작
DELETE FROM emp WHERE empno = 8011 AND ename = '장길산';

SELECT * FROM emp;

ALTER TABLE emp ADD CONSTRAINT pk_emp PRIMARY KEY(empno);



In [ ]:
-- primary key 추가 삭제 - mysql
-- alter 수정할 때
-- primary key 삭제명령어 - mysql만 가능 
alter table 테이블명 drop primary key;
-- primary key 추가 명령어 - mysql만 
alter talble 테이블명 add constraint 키이름 primary key(필드명);


### 🟡 foreign key (외부키)
- 테이블과 테이블의 관계성에 입각해서 상호 제약을 가하고자 할 때.
- 테이블을 나누는 구조 (제 1 정규화, 제 2 정규화, BNF 정규화)
- 실무에서 바라볼 때 1 : N의 관계
- 고용인, 부서테이블 
  - 각 직원마다 부서명을 따로 저장하게되면 메모리가 많이 든다.
  - 부서명 바꿀 때마다 직원마다 부서명 바꾸러 다닌다. (오버헤드 (쓸 때 없는 일))
    - 만약 한명 빼고 바꾸면 이 사람은 부서가 없는 사람이 된다.
  - 일관성이 없다.
  - 테이블을 나누게 되면 -> 메모리 부족 해결, 부서코드만 저장하여 중복을 줄임.

- emp 테이블의 depno |  dept 테이블의 deptno (primary 또는 unique)
  - emp 테이블에서 forign key를 dep 테이블에 걸어준다.
  - emp 테이블의 deptno 필드에는 dept테이블의 deptno 필드 내에 있는 값만 엏을 수 있다. (부서번호가 10, 20, 30, 40)
  - emp 테이블의 부서번호 50 넣으면 그런 부서번호가 없다고 에러번호가 뜹니다.
  - dept 테이블 입장에서는 부서번호 삭제 불가능  ->  이미 사용중인 부서번호 삭제 불가

| 항목        | 설명                                                                                                                         |
| --------- | -------------------------------------------------------------------------------------------------------------------------- |
| 정의        | 한 테이블이 다른 테이블의 **기본 키(PK)** 또는 \*\*고유 키(UNIQUE)\*\*를 참조하여 관계를 맺는 제약조건                                                      |
| 목적        | <ul><li>**테이블 간 관계 형성** (예: 고용인 ↔ 부서)</li><li>**데이터 무결성 보장** (존재하지 않는 값 삽입 방지)</li></ul>                                   |
| 정규화와의 관련성 | <ul><li>**1NF, 2NF, BCNF** 정규화 과정에서 테이블을 분리함</li><li>분리된 테이블 간 관계를 Foreign Key로 연결</li></ul>                               |
| 관계 예시     | 실무에서 자주 사용하는 **1\:N 관계**<br>예: 하나의 부서(1)에 여러 명의 직원(N) 소속                                                                   |
| 외래 키 조건   | <ul><li>외래 키는 반드시 참조 대상 테이블의 **기본 키(PK)** 또는 \*\*고유 키(UNIQUE)\*\*를 참조해야 함</li><li>참조 대상 테이블에 존재하지 않는 값을 입력할 수 없음</li></ul> |

##### ⚫️ 실무 예시: 직원(emp) 테이블과 부서(dept) 테이블
| 테이블    | 컬럼명           | 설명                        |
| ------ | ------------- | ------------------------- |
| `dept` | `deptno` (PK) | 부서 번호 (기본 키)              |
| `emp`  | `deptno` (FK) | 외래 키: 부서 테이블의 `deptno` 참조 |




In [ ]:
-- < foreign key (외부키) >
DESC dept;
ALTER TABLE 테이블명
ADD CONSTRAINT 외부키 이름
FOREIGN KEY (필드명)
REFERENCES 참조테이블명(참조필드명)
ON DELETE CASCADE  -- 부모 레코드 삭제 시 자식도 삭제
ON UPDATE CASCADE  -- 부모키값 변경 시 자식도 자동으로 변경

SELECT * FROM dept;

-- 1. 참조하는 테이블(dep)의 dept가 반드시 primary거나 unique조건을 만족해야 한다.
-- 2. 데이터 타입이 동일 해야한다.

ALTER TABLE emp ADD constraint fk_emp_dept
FOREIGN KEY (deptno) 
references dept(deptno);
-- 테이블 상호간에 제약조건이 방생한다.
-- SHOW CREATE TABLE emp\G;

SELECT * FROM dept;
DELETE FROM dept WHERE deptno = 10;  -- 오류가 맞음 / 외부키 때문에 삭제 불가



-- 홍길동한테 부서번호 부여하기 
SELECT * FROM emp;
UPDATE emp SET deptno =50 WHERE empno=8000;  -- 오류 나는 것이 맞음 

<br><br><br><br>

# 🟩 join

- JOIN은 두 개 이상의 테이블을 공통된 컬럼을 기준으로 연결하여 조회할 때 사용합니다.
- inner / outer / full - ansi 표준
- emp 테이블에는 부서번호만 있다.
- dept 테이블에는 부번호와 부서명을 알고 싶다.
- 두개 이상의 테이블을 묶어서 새로운 정보를 창출한다.


In [ ]:
-- ---------------------------------
-- < inner join > : 두 테이블에서 서로 공통된 조건에 맞는 행(row)만 결합해서 보여주는 JOIN
-- 이거는 query 표준이 아니기 때문에 다른 DBMS에서는 에러가 날 수 있다. (oracle, mysql만 가)
SELECT A.empno, A.ename, A.demtno, B.dname
FROM emp A, dept B
WHERE A.deptno = B.deptno;   -- JOIN 조건이 동등조건  (equl 조건)


-- 표준 join 
SELECT A.empno, A.ename, A.demtno, B.dname
FROM emp A
INNER JOIN dept B ON A.deptno = B.deptno

SELECT * FROM emp;


-- 회원번호가 7566, 7788, 7902, 7934, 8000 
SELECT A.empno, A.ename, A.deptno, B.dname
FROM emp A, dept B
WHERE  A.deptno = B.deptno AND
A.empno IN (7566, 7788, 7902, 7934, 8000);
-- 단점 : 조인 조건과 검색조건이 구분이 안간다. 그래서 조인이 여러번 이뤄질 경우에 보기 안좋다.


-- inner join 은 양쪽 테이블에 값이 존재할 때 된다.
SELECT A.empno, A.ename, A.deptno, B.dname
FROM emp A
INNER JOIN dept B ON A.deptno = B.deptno
WHERE A.empno IN (7566, 7788, 7902, 7934, 8000);


-- ---------------------------------
-- < outer join > : left, right
-- from 절에 가까운 테이블 left (left 테이블의 데이터 다출력 )
-- form 절로부터 먼 테이블이 right  (right 테이블의 데이터 다 출력 )
SELECT A.empno, A.ename, A.deptno, B.dname
FROM emp A
LEFT OUTER JOIN dept B ON A.deptno = B.deptno;
-- GPT : 둘 다 한쪽 테이블의 모든 데이터는 무조건 출력하고, 다른 쪽 테이블의 데이터는 조건에 맞는 것만 출력


-- ---------------------------------
-- 중복성 배제  
-- right outer join
SELECT DISTINCT deptno FROM emp;

SELECT A.empno, A.ename, A.deptno, B.dname
FROM emp A
right OUTER JOIN dept B ON A.deptno = B.deptno;
-- 40번이 할당을 못 받은 것으로 나옵니다.



In [ ]:
-- ---------------------------------
-- < join practice >
-- actor_id, first_name, last_name
USE sakila;

SELECT * FROM actor LIMIT 10;
SELECT * FROM film LIMIT 10;
-- 배우가 누군지 알 수가 없다... 


-- 어떤 배우가 어떤 영화에 출연했는지 알고 싶다...
SELECT * FROM film_actor LIMIT 10;


-- 영화 필름 기준으로 join 
SELECT title, description, B.actor_id,
concat(C.last_name, '', C.first_name) actor_name
FROM film A
LEFT OUTER JOIN film_actor B ON A.film_id = B.film_id
LEFT OUTER JOIN actor C ON B.actor_id = C.actor_id



-- ============================================
-- 문제 1
-- category가 conmedy인 영화 목록만
-- film, category, film_category

SELECT * FROM category LIMIT 20;  -- 5
SELECT * FROM film_category LIMIT 20;
SELECT * FROM film LIMIT 20;

-- 시도 1
SELECT A.category_id, B.category_id 
FROM category A
LEFT OUTER JOIN film_category B ON A.category_id = B.category_id;
-- LEFT OUTER JOIN actor C ON B.actor_id = C.actor_id

-- 시도2
SELECT A.category_id, B.category_id 
FROM category A
LEFT OUTER JOIN film_category B ON A.category_id = B.category_id
LEFT OUTER JOIN film C ON B.category_id = C.film_id
WHERE A.name = 'Comedy';

-- 같이 풀어보기 
SELECT title 
FROM film A
LEFT OUTER JOIN film_category B ON A.film_id = B.film_id
LEFT OUTER JOIN category C ON B.category_id = C.category_id
WHERE C.name = 'Comedy';



-- ============================================
-- 문제 2
-- 고객의 이름과 고객이 대여 한 영화 제목을 모두 출력하자 
-- customer, rental, invetory, film
-- inventory_id, store_id, film_id

-- !!!! 방법을 알아냈습니다. 이런 규칙을 왜 안알려주는지 모르겠지만....
SELECT * FROM customer LIMIT 20;     -- 1. 먼저 구할 값을 정한다.
		-- 2. 관련 있는 데이터들의 공통점을 파악한다. 그래서 다 가져온다. (like rental, inventory, film)
SELECT * FROM rental LIMIT 20;     -- 3. 
SELECT * FROM inventory LIMIT 20;
SELECT * FROM film LIMIT 20;

 -- 내가 직접 풀어보기! (성공)
SELECT A.first_name, A.last_name, B.rental_date, D.title
FROM customer A
LEFT OUTER JOIN rental B ON A.customer_id = B.customer_id
LEFT OUTER JOIN inventory C ON B.inventory_id = C.inventory_id
LEFT OUTER JOIN film D ON C.film_id = D.film_id
LIMIT 10;

-- 같이 풀어보기 (concat을 추가하고 테이블 명치까지변경하는 방법을 알게 되었습니다.)
SELECT concat(A.first_name, "  ", A.last_name) name, B.rental_date, D.title
FROM customer A
LEFT OUTER JOIN rental B ON A.customer_id = B.customer_id
LEFT OUTER JOIN inventory C ON B.inventory_id = C.inventory_id
LEFT OUTER JOIN film D ON C.film_id = D.film_id
LIMIT 10;



-- ========================================================
-- 문제 3 : NICK WAHLBERG 라는 배우가 출연한 영화의 제목 조회하기

SELECT * FROM actor LIMIT 20;
SELECT * FROM film_actor LIMIT 20;
SELECT * FROM film LIMIT 20;


SELECT concat(A.first_name, "  ", A.last_name) name, C.title
FROM actor A
LEFT OUTER JOIN film_actor B ON A.actor_id = B.actor_id
LEFT OUTER JOIN film C ON B.film_id = C.film_id
WHERE A.first_name = 'NICK' AND A.last_name = 'WAHLBERG'
LIMIT 10;

-- 같이 풀어보기 ( inner join으로도 할 줄 알아야 합니다.)
SELECT CONCAT(A.first_name, ' ', A.last_name) AS name, C.title
FROM actor A
INNER JOIN film_actor B ON A.actor_id = B.actor_id
INNER JOIN film C ON B.film_id = C.film_id
WHERE A.first_name = 'NICK' AND A.last_name = 'WAHLBERG'
LIMIT 10;



-- ========================================================
-- 문제 4 : London 도시의 고객 이름만 출력 

SELECT * FROM customer LIMIT 20;
SELECT * FROM address LIMIT 20;
SELECT * FROM city LIMIT 20;

SELECT concat(A.first_name, "  ", A.last_name) name, C.city
FROM customer A
LEFT OUTER JOIN address B ON A.address_id = B.address_id
LEFT OUTER JOIN city C ON B.city_id = C.city_id
WHERE C.city = 'London'

-- 같이 풀어보기
SELECT concat(A.first_name, "  ", A.last_name) name, C.city
FROM customer A 
JOIN address B ON A.address_id = B.address_id
JOIN city C ON B.city_id = C.city_id
WHERE C.city = 'London'





<br><br><br><br>

# 🟩 Subquery (서브쿼리)

- 서브쿼리(Subquery)란, 쿼리 안에 포함된 또 다른 쿼리
- 주로 괄호 () 안에 들어가며, **메인 쿼리(main query)**에서 결과를 활용


| 종류              | 위치      | 설명        | 예시                                               |
| ---------------- | -------- | ---------- | ------------------------------------------------- |
| **스칼라 서브쿼리**    | SELECT 절 | 한 개의 값을 반환 | `SELECT (SELECT MAX(salary) FROM emp)`         |
| **인라인 뷰**        | FROM 절   | 테이블처럼 사용   | `FROM (SELECT * FROM emp WHERE deptno=10) AS sub` |
| **서브쿼리 WHERE 절** | WHERE 절  | 조건에 따라 필터링 | `WHERE empno IN (SELECT empno FROM ...)`         |
| **EXISTS 서브쿼리**  | WHERE 절  | 존재 여부만 확인  | `WHERE EXISTS (SELECT * FROM ...)`              |


- join 속도를 빠르게 하려면, join 필드에 인덱스를 만들어줘야 한다.

- subQuery : 서브쿼리는 주 쿼리 옆에서 주워리보다 먼저 실행되서 결과를 가져온 다음에 주쿼리가 실행된다.

- 서브쿼리는 select 절, from 절, where, order, by 절 등 ek rksdmgkek.
  - select 절
    - 스칼라 서브 쿼리, 결과값이 null이거나 한개만 가져오는 쿼리
    - join을 대체할 수 있다. 우리가 볼 떄 편해보임, 조인이 일반적으로 더 빠르다.
    - 가급적 join으로 해결하고, join 안될 경우 서브쿼라를 사용하자!


In [ ]:
-- =============================================
-- 스칼라 서브쿼리 - select 

USE mydb;

-- 사원 번호, 사원 이름, 부서명을 가져오려고 한다.
SELECT empno, ename, deptno
FROM emp;

SELECT dname, deptno FROM dept WHERE deptno = 10;

SELECT * FROM address LIMIT 20;
SELECT * FROM city LIMIT 20;

-- SELECT empno, ename, deptno,
-- (SELECT dname, deptno FROM dept WHERE deptno = 10) AS dname
-- FROM emp;


SELECT empno, ename, deptno,
(SELECT dname FROM dept B WHERE A.deptno = B.deptno) AS dname
FROM emp A;

-- --------------------------------

USE sakila;

SELECT * FROM actor LIMIT 20;
SELECT * FROM film_actor LIMIT 20;
SELECT * FROM film LIMIT 20;

SELECT A.first_name, A.last_name, B.film_id
FROM actor A
LEFT OUTER JOIN film_actor B ON A.actor_id = B.actor_id;

-- join + 서브쿼리 select절 사용하기   (굳이 이렇게 사용하지 않습니다.)
SELECT A.first_name, A.last_name, B.film_id,
(SELECT title FROM film C WHERE B.film_id = C.film_id) AS title
,(SELECT length FROM film C WHERE B.film_id = C.film_id) AS length
FROM actor A
LEFT OUTER JOIN film_actor B ON A.actor_id = B.actor_id;




-- =============================================
-- from 절 : 다중행을 반환한다. 다중행 서브쿼리, 인라인뷰 

USE mydb;

SELECT * FROM dept LIMIT 10;  -- dname
SELECT * FROM emp LIMIT 10;  -- ename

SELECT * FROM emp WHERE deptno IN (10, 20);  -- 여기에서 사원이름, 부서명만 나오게 하고싶다. 

SELECT A.ename, dname
FROM (
							SELECT * FROM emp WHERE deptno IN (10, 20)
) AS A
JOIN dept B ON A.deptno = B.deptno			




-- =============================================
-- inline view

USE sakila;

-- film_id : ambigous
SELECT A.film_id, title, length, actor_id
FROM film A
LEFT OUTER JOIN film_actor B ON A.film_id = B.film_id;

-- ! 인라인 뷰 !!!!!
SELECT first_name, last_name, title
FROM actor A
LEFT OUTER JOIN (
						SELECT A.film_id, title, length, actor_id
						FROM film A
						LEFT OUTER JOIN film_actor B ON A.film_id = B.film_id
						-- 인라인뷰라고 부른다.
) B ON A.actor_id = B.actor_id




-- =============================================
-- where 절 

USE mydb;

-- emp 테이블에 smith라는 사람의 부서와 같은 부서에 있는 사람들 
SELECT * FROM dept LIMIT 10;  -- dname
SELECT * FROM emp LIMIT 21;  -- ename

SELECT deptno FROM emp WHERE ename ='ALLEN';


SELECT * FROM emp WHERE deptno = 20;

-- 여기 이렇게 했습니다. 
SELECT * FROM emp 
WHERE deptno = (SELECT deptno FROM emp WHERE ename = 'ALLEN');



-- =============================================
-- 같이푸는 문제: 부서 평균 급여보다 급여가 많은 사원 조회

SELECT * FROM emp LIMIT 10;

SELECT * FROM emp 
WHERE sal > (SELECT avg(sal) FROM emp); 



-- =============================================
-- 같이 푸는 문제: ALLEN 가 근무하는 부서의 급여 평균보다 급여를 받는 사람들 정보를  확인하고자 한다.

SELECT avg(sal) FROM emp WHERE deptno = (SELECT deptno FROM emp WHERE  ename = 'ALLEN');

-- 3중 쿼리 되어버림...
SELECT ename, sal FROM emp
WHERE sal > (SELECT avg(sal) 
														FROM emp WHERE deptno = (SELECT deptno FROM emp WHERE ename = 'ALLEN'));




-- =============================================
-- 문제 1

-- 가장 높은 급여를 받는 사원 정보 조회하기 MAX
SELECT * FROM emp LIMIT 20;


SELECT * FROM emp WHERE sal = MAX(sal); 

SELECT ename, sal FROM emp 
WHERE sal = (SELECT MAX(sal) FROM emp);  


-- 매니저가 존재하는 원만 조회 
-- 나의 mgr 필드가 emp 테이블에 존재하느냐???
SELECT * 
FROM emp 
WHERE mgr IS NOT NULL;

SELECT * 
FROM emp 
WHERE mgr IN (SELECT empno FROM emp);



-- ============================================
-- 기존 JOIN 문제를 서브쿼리로 바꾸는 문제 
-- ============================================
-- 문제 2
-- 고객의 이름과 고객이 대여 한 영화 제목을 모두 출력하자 
-- customer, rental, invetory, film
-- inventory_id, store_id, film_id

USE sakila;

SELECT * FROM customer LIMIT 20;   
SELECT * FROM rental LIMIT 20;     -- 3. 
SELECT * FROM inventory LIMIT 20;
SELECT * FROM film LIMIT 20;

 -- 내가 직접 못 풀었

-- 같이 풀어보기 (concat을 추가하고 테이블 명치까지변경하는 방법을 알게 되었습니다.)
SELECT concat(A.first_name, "  ", A.last_name) name,
	(SELECT title from film D where C.film_id = D.film_id) AS title
FROM customer A
LEFT OUTER JOIN rental B ON A.customer_id = B.customer_id
LEFT OUTER JOIN inventory C ON B.inventory_id = C.inventory_id
left outer join film D on D.film_id=C.film_id
LIMIT 10;




-- ========================================================
-- 문제 3 : NICK WAHLBERG 라는 배우가 출연한 영화의 제목 조회하기

/*
SELECT concat(A.first_name, "  ", A.last_name) name, C.title
FROM actor A
LEFT OUTER JOIN film_actor B ON A.actor_id = B.actor_id
LEFT OUTER JOIN film C ON B.film_id = C.film_id
WHERE A.first_name = 'NICK' AND A.last_name = 'WAHLBERG'
LIMIT 10;
*/

USE sakila

SELECT * FROM actor LIMIT 20;
SELECT * FROM film_actor LIMIT 20;
SELECT * FROM film LIMIT 20;

-- 같이 풀어보기 
SELECT (SELECT title FROM film C WHERE B.film_id = C.film_id) AS title
FROM actor AS 
INNER JOIN film_actor B ON A.actor_id = B.actor_id
WHERE A.first_name='NICK' and A.last_name='WAHLBERG';



SELECT CONCAT(A.first_name, ' ', A.last_name) AS name, C.title
FROM actor A
JOIN film_actor B ON A.actor_id = B.actor_id
JOIN film C ON B.film_id = C.film_id
WHERE A.actor_id = (
  SELECT actor_id
  FROM actor
  WHERE first_name = 'NICK' AND last_name = 'WAHLBERG'
  LIMIT 1
)
LIMIT 10;




## 🟢 Correlated Subquery (상관 서브쿼리)

- 상관쿼리 : 외부 쿼리의 값을내부 쿼리에서 참조하는 서브쿼리를 말한다.
- 외부 쿼리의 각 행마다 내부 쿼리가 실행되는 구조

- exists, any, all, in 등이 있다.

  - exists : 서브쿼리의  실행결과가 하나라고 있으면 True  / 1개라도 없으면 False 반환
    - 서브쿼리의 싱행 결과가 한건이라도 있으면 실행된다.  

  - Any
    - 조건을 만족하는게 하나라도 있으면 수행 
    - 부등호 or 부등호 or 부등호 or 부등호 or 부등호

  - all
    - 모든 조건을 만족하는 부등호 and 부등호 and 부등호 and 부등호 and

  - in 
    - 등호 or 등호 or 등호 or

In [ ]:
-- 부서별 평균 급여보다 높은 사원조회
-- 부서별 평균값이 필요
USE mydb;

SELECT avg(sal) FROM emp WHERE deptno = 10;
SELECT avg(sal) FROM emp WHERE deptno = 20;
SELECT avg(sal) FROM emp WHERE deptno = 30;
SELECT avg(sal) FROM emp WHERE deptno = 40;

SELECT empno, sal, deptno FROM emp;

SELECT empno, sal, deptno FROM emp
WHERE sal > (SELECT avg(sal)FROM emp WHERE deptno=10);

-- 이것을 상관쿼리라고 부릅니다. 
-- 외부의 A의 SubQuey(내부쿼리 )의 deptno가 서로 관계가 있었다.
SELECT empno, sal, deptno FROM emp A
WHERE sal > (SELECT avg(sal)FROM emp B WHERE B.deptno=A.deptno);




-- exists : 매니저가 존재하는 사원만 조회하려고 한다.

SELECT empno, ename, mgr FROM emp;

SELECT ename FROM emp WHERE mgr=7788;

-- 서브쿼리의 실행결과가 하나라도 있으면 외부쿼리를 실행한다.
-- SELECT	* FROM emp WHERE EXISTS (서브쿼리)  -- 서브쿼리의 결과 유무만 따집니다.

SELECT empno, ename, mgr FROM emp
WHERE EXISTS (
				SELECT ename FROM emp WHERE mgr=7788
);


-- 상관 쿼리 
SELECT empno, ename, mgr FROM emp A
WHERE EXISTS (
				SELECT 1 FROM emp B WHERE A.mgr = B.empno
);

<br><br><br><br>

# 🟩 w3schools DB 예제


In [ ]:
USE w3schools;
-- https://www.w3schools.com/ 

-- 확인하기 
SELECT * FROM orders LIMIT 10;

-- 주문내역, 고객이름, 판매자 이름 까지만 출력해봐라

SELECT * FROM Orders
LIMIT 10;
SELECT * FROM EMPLOYEES
LIMIT 10;
SELECT * FROM customers
LIMIT 10;
SELECT * FROM OrderDetails
LIMIT 10;

-- 직접 풀어보기 
-- SELECT CstomerName, title
-- FROM ORDERS A
-- JOIN film_actor B ON A.actor_id = B.actor_id
-- JOIN film C ON B.film_id = C.film_id
-- WHERE A.first_name = 'NICK' AND A.last_name = 'WAHLBERG'
-- LIMIT 10;

-- 같이 풀어보기 
SELECT A.*, B.customerName, concat(C.lastName, "  ", C.firstName) AS EmployeeName
FROM orders A
JOIN customers B ON A.customerId = B.customerId
JOIN EMPLOYEES C ON A.employeeId = C. employeeId;

-- 고치기 
SELECT A.orderID, B.customerName, concat(C.lastName, "  ", C.firstName) AS EmployeeName
FROM orders A
JOIN customers B ON A.customerId = B.customerId
JOIN EMPLOYEES C ON A.employeeId = C. employeeId;



-- ===========================================
-- join 안쓰고 서브쿼리로 바꾸기

SELECT orderId,
	(SELECT customerId = B.customerId FROM customers B
		WHERE A.customerId = B.customerId) AS customerName
,
(SELECT concat(C.lastName, "  ", C.firstName)
FROM employees C
WHERE  A.employeeId = C.employeeId) AS employeeName
FROM orders A;


-- chatGPT 제시한 내용  
SELECT 
  A.orderId,

  		-- 고객 이름 가져오기
  (SELECT B.customerName 
   FROM customers B
   WHERE A.customerId = B.customerId) AS customerName,

  		-- 직원 이름 (성 + 이름) 가져오기
  (SELECT CONCAT(C.lastName, ' ', C.firstName)
   FROM employees C
   WHERE A.employeeId = C.employeeId) AS employeeName

FROM orders A;

-- 리눅스 MySQL 대소문자를 따집니다.


<br><br><br><br>

# 🟩 Group By

In [ ]:
-- 주의사항 :  리눅스 MySQL은 필드명이나 테이븡명의 대소문자를 따진다.

-- 그룹함수, avg, max, min, count, sum ....

USE mydb;

-- 에러 예시 
SELECT ename, sal, avg(sal) FROM emp LIMIT 10;
-- 이 코드는 에러가 납니다. 
-- 이유는 avg 내장함수는 ename, sal 보다 갯수가 적기 때문에 같이 나올 수가 없다.


-- 이럴 때 서브쿼리를 쓸 수 있습니다.
SELECT ename, sal,
(SELECT avg(sal) FROM emp) as avg_sal
FROM emp;

-- 부분합, 그룹별로 묶는게 가능하다.
-- 각 부서별로 급여 평균 을 확인하고 싶다.
-- 그룹함수는 group by 절에 온 필드는 사용가
SELECT deptno, avg(sal)
FROM emp
GROUP BY deptno;


-- 이름과 부서번호 급여 부서별 평균을 확인하고 싶다.
SELECT 
			ename, 
			deptno, 
			(SELECT avg(sal) FROM emp B WHERE A.deptno = B.deptno) AS dept_sal
FROM emp A;


-- 서브쿼리와 join 합치
SELECT ename, A.deptno
FROM emp A
LEFT OUTER JOIN
			(SELECT deptno, avg(sal) dept_sal, sum(sal) sum_sal, max(sal) max_sal, min(sal) min_sal
				FROM emp 
				GROUP BY deptno) B
ON A.deptno = B.deptno



-- orders 테이블에서 고객별 주문 개수를 구하고, 정렬 주문수가 많은 고객부터 내림차순
-- 고객이름, 주문카운트

USE w3schools;

SELECT * FROM Orders LIMIT 10;
SELECT * FROM EMPLOYEES LIMIT 10;
SELECT * FROM customers LIMIT 10;
SELECT * FROM OrderDetails LIMIT 10;


-- 같이 풀어보기 
SELECT customerId, count(customerId)
FROM orders
GROUP BY customerId
ORDER BY count(customerId) DESC;
-- where 절에는 group 함수 사용 못함.


-- 구매개수가 5개 이상만  
SELECT customerId, count(customerId)
FROM orders
GROUP BY customerId
-- WHERE count(customerId) >= 5      -- [중요]  where 절에는 group 함수 사용 못함.
HAVING count(customerId) >= 5          -- 순서도 중요합니다.  GROUP BY  =>  HAVING
ORDER BY count(customerId) DESC;

